In [ ]:
!pip install transformers sentencepiece datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import tensorflow as tf
import transformers
import numpy as np
import pandas as pd

from transformers import AutoTokenizer, TFRobertaForMaskedLM
from datasets import Dataset

In [ ]:
RANDOM_SEED = 1337

transformers.set_seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [ ]:
MODEL_NAME = "roberta-large"

narration_mode = True

if narration_mode:
   # Training model on Narration scenarios:
  selected_scenarios = ["Cinderella", "Sandwich"]
  MODEL_MODE = "-aphasia-narration-weightdecay0-lr2e4"
  EPOCHS = 10
else:
   # Training model on Picture description scenarios:
  selected_scenarios = ["Window", "Cat", "Umbrella", "Flood"]
  MODEL_MODE = "-aphasia-picture-description"
  EPOCHS = 10

In [ ]:
protocol_control_df = pd.read_csv("protocol_control_preprocessed.csv",index_col=False)
protocol_control_df['word_count'] = protocol_control_df['preprocessed_text'].str.split().str.len()
protocol_control_df = protocol_control_df.loc[(protocol_control_df['word_count'] > 2)]

protocol_control_df = protocol_control_df[protocol_control_df.scenario.isin(selected_scenarios)]

In [ ]:
print(len(protocol_control_df))
protocol_control_df = protocol_control_df[protocol_control_df["preprocessed_text"].str.contains("@") == False]
print(len(protocol_control_df))


14724
14710


In [ ]:
protocol_control_df.head()

,Unnamed: 0,scenario,source_file,original_text,preprocessed_text,word_count
44,213,Cinderella,capilouto01a.txt,once upon a time &-uh there was a little girl ...,once upon a time there was a little girl named...,12
45,216,Cinderella,capilouto01a.txt,and &-um her mother passed away .,and her mother passed away .,6
46,219,Cinderella,capilouto01a.txt,and then eventually her father married again .,and then eventually her father married again .,8
47,222,Cinderella,capilouto01a.txt,and he didn't realize that the woman he marrie...,and he did not realize that the woman he marri...,14
48,225,Cinderella,capilouto01a.txt,but then the father passed away and so Cindere...,but then the father passed away and so Cindere...,16


In [ ]:
protocol_control_df['word_count'].max()

52

In [ ]:
len(protocol_control_df)

14710

In [ ]:
train_df = protocol_control_df.sample(frac=0.80,random_state=RANDOM_SEED)
dev_df = protocol_control_df.drop(train_df.index)

protocol_control_train_df = Dataset.from_pandas(train_df)
protocol_control_dev_df = Dataset.from_pandas(dev_df)

protocol_control_train_df = protocol_control_train_df.flatten()
protocol_control_dev_df = protocol_control_train_df.flatten()

print(len(train_df))
print(len(dev_df))

11768
2942


In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
def preprocess_function(examples):
    return tokenizer(["".join(x) for x in examples["preprocessed_text"]], max_length=128, truncation=True)

tokenized_control_dataset_train = protocol_control_train_df.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=protocol_control_train_df.column_names,
)

tokenized_control_dataset_dev = protocol_control_train_df.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=protocol_control_dev_df.column_names,
)

Map (num_proc=4):   0%|          | 0/11768 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/11768 [00:00<?, ? examples/s]

In [ ]:
block_size = 128

def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_dataset_train = tokenized_control_dataset_train.map(group_texts, batched=True, num_proc=4)
lm_dataset_dev = tokenized_control_dataset_dev.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/11768 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/11768 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15, return_tensors="tf")

In [ ]:
from transformers import create_optimizer, AdamWeightDecay
optimizer = AdamWeightDecay(learning_rate=2e-4, weight_decay_rate=0)

In [ ]:
model = TFRobertaForMaskedLM.from_pretrained(MODEL_NAME)

tf_train_set = model.prepare_tf_dataset(
    lm_dataset_train,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_dev_set = model.prepare_tf_dataset(
    lm_dataset_dev,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForMaskedLM: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForMaskedLM for predictions without further training.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to enc

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir=MODEL_NAME + MODEL_MODE + "_eps_" + str(EPOCHS) ,
    tokenizer=tokenizer,
)

model.compile(optimizer=optimizer)
model.fit(x=tf_train_set, validation_data=tf_dev_set, epochs=EPOCHS, callbacks=[callback])

Cloning https://huggingface.co/Middelz2/roberta-large-aphasia-narration-weightdecay0-lr2e4_eps_10 into local empty directory.


Epoch 1/10
75/75 [==============================] - 213s 3s/step - loss: 2.1420 - val_loss: 1.6450
Epoch 2/10
75/75 [==============================] - 302s 4s/step - loss: 1.6971 - val_loss: 1.4556
Epoch 3/10
75/75 [==============================] - 298s 4s/step - loss: 1.6025 - val_loss: 1.3882
Epoch 4/10
75/75 [==============================] - 307s 4s/step - loss: 1.4763 - val_loss: 1.2997
Epoch 5/10
75/75 [==============================] - 303s 4s/step - loss: 1.4301 - val_loss: 1.3055
Epoch 6/10
75/75 [==============================] - 316s 4s/step - loss: 1.4358 - val_loss: 1.3317
Epoch 7/10
75/75 [==============================] - 321s 4s/step - loss: 2.2816 - val_loss: 2.4774
Epoch 8/10
75/75 [==============================] - 314s 4s/step - loss: 2.7754 - val_loss: 2.0994
Epoch 9/10
75/75 [==============================] - 312s 4s/step - loss: 4.5272 - val_loss: 5.5713
Epoch 10/10
75/75 [==============================] - 313s 4s/step - loss: 5.6128 - val_loss: 5.5802


# Evaluation of model using Perplexity



In [ ]:
import math

eval_results = model.evaluate(tf_dev_set)
print(f"Perplexity: {math.exp(eval_results):.2f}")


76/76 [==============================] - 42s 548ms/step - loss: 0.9156
Perplexity: 2.50


In [ ]:
eval_results

0.9156346321105957

In [ ]:
math.exp(0.9156)

2.4982737665654735